In [1]:
import numpy as np
import pandas as pd 
from tqdm import tqdm

In [2]:

# Define file path names
inputFileName = '/home/jovyan/ODBiz/1-PreProcessing/processed/BC_Vancouver_Business_Licences.csv'
outputFileName = '/home/jovyan/ODBiz/1-PreProcessing/vancouver_dedup/BC_Vancouver_Business_Licences.csv'

# Load in variablemap to help with column names
# var_map = pd.read_csv('/home/jovyan/ODBiz/2-OpenTabulate/variablemap.csv')
# var_map = var_map.set_index('localfile')
# street_no = var_map.loc['BC_Vancouver_Business_Licences.csv', 'street_no']

# Load in the csv
total_lines = 636855
chunksize = 10000
# types_dict = {street_no: str} # Read in street numbers as strs since some of them are formatted weirdly
types_dict = str
df = pd.concat([chunk for chunk in tqdm(pd.read_csv(inputFileName, chunksize=chunksize, dtype = types_dict), desc='Loading data', total=total_lines//chunksize+1)])
df = df.fillna('')

# Display first few rows of df
pd.options.display.max_columns = None
df.head()

Loading data: 100%|██████████| 64/64 [00:05<00:00, 12.28it/s]


,Unnamed: 0,FOLDERYEAR,LicenceRSN,LicenceNumber,LicenceRevisionNumber,BusinessName,BusinessTradeName,Status,IssuedDate,ExpiredDate,BusinessType,BusinessSubType,Unit,UnitType,House,Street,City,Province,Country,PostalCode,LocalArea,NumberofEmployees,FeePaid,ExtractDate,Geom,long,lat,business_name
0,0,13,1988567,13-325119,0,Beyond Sound Event Productions Ltd,Beyond Sound,Issued,2013-03-15T17:56:12-04:00,2013-12-31,Entertainment Services,Disc Jockey,106+107,Unit,470,KINGSWAY,Vancouver,BC,CA,V5T 3J9,Mount Pleasant,1.0,158.0,2019-07-21T16:49:09-04:00,"{""coordinates"": [-123.093530332101, 49.2576936...",-123.093530332101,49.2576936787665,Beyond Sound Event Productions Ltd
1,1,13,1988589,13-325138,0,J Robert Thibodeau Architecture + Design Inc,,Issued,2013-05-02T13:30:53-04:00,2013-12-31,Office,Architect,,,138,W 8TH AV,Vancouver,BC,CA,V5Y 1N2,Mount Pleasant,7.0,147.0,2019-07-21T16:49:09-04:00,"{""coordinates"": [-123.108309208116, 49.2635872...",-123.108309208116,49.2635872409021,J Robert Thibodeau Architecture + Design Inc
2,2,13,1988600,13-325148,0,(Narinder Gill),,Issued,2013-03-15T17:52:44-04:00,2013-12-31,Secondary Suite - Permanent,,,,,,Vancouver,BC,CA,,Sunset,0.0,97.0,2019-07-21T16:49:09-04:00,,,,(Narinder Gill)
3,3,13,1988632,13-187211,1,MLF Event Architecture Inc,,Issued,2013-03-19T18:17:17-04:00,2013-12-31,Office,Architect,215,Unit,1080,MAINLAND ST,Vancouver,BC,CA,V6B 2T4,Downtown,1.0,22.0,2019-07-21T16:49:09-04:00,"{""coordinates"": [-123.120093418192, 49.2755430...",-123.120093418192,49.2755430466569,MLF Event Architecture Inc
4,4,13,1988768,13-186372,1,Ocean West Financial Group Inc,,Issued,2013-03-26T18:09:00-04:00,2013-12-31,Office,Insurance Company,1507,Unit,1030,W GEORGIA ST,Vancouver,BC,CA,V6E 2Y3,West End,6.0,22.0,2019-07-21T16:49:09-04:00,"{""coordinates"": [-123.122008869142, 49.2845352...",-123.122008869142,49.2845352944803,Ocean West Financial Group Inc


In [30]:
# Define columns to identify duplicates
dup_keys = ['business_name',
            'province',
            'business_sector',
            'unit',
            'street_no',
            'street_name',
            'city',
            'province',
            'postal_code',
            'country'
            ]

# Load in variablemap to help with column names
var_map = pd.read_csv('/home/jovyan/ODBiz/2-OpenTabulate/variablemap.csv')
var_map = var_map.set_index('localfile')
dup_keys_van = var_map.loc['BC_Vancouver_Business_Licences.csv', dup_keys].to_dict()
print(dup_keys_van.values())

dict_values(['BusinessName', 'Province', 'BusinessType', 'Unit', 'House', 'Street', 'City', 'PostalCode', 'Country'])


In [5]:
# Create column concatenating dup_keys columns
df['bus_idx'] = ''
# for row in tqdm(df.itertuples(), total = total_lines):
for v in tqdm(dup_keys_van.values()):
    val = df[v]
    # val = getattr(row, v)
    # if val is not np.nan:
    # i = row.Index
    # print('')
    # print(i)
    # print(val)
    df['bus_idx'] += val
    df['bus_idx'] += '_'


100%|██████████| 9/9 [00:03<00:00,  2.74it/s]


In [20]:
# Sort by time, most recent first
df_sorted = df.sort_values(by = ['bus_idx', 'FOLDERYEAR', 'IssuedDate'], ascending=False)
time_sorted_path = '/home/jovyan/ODBiz/1-PreProcessing/vancouver_dedup/time_sorted.csv'
df_sorted.to_csv(time_sorted_path, index = False)
print(f'File saved to {time_sorted_path}')

In [21]:
# Deduplicate using the keys
df_dedup = df_sorted.drop_duplicates('bus_idx')
df_dedup_path = '/home/jovyan/ODBiz/1-PreProcessing/vancouver_dedup/Vancouver_dedup.csv'
df_dedup.to_csv(df_dedup_path, index = False)
print(f'File saved to {df_dedup_path}')

File saved to /home/jovyan/ODBiz/1-PreProcessing/vancouver_dedup/Vancouver_dedup.csv


In [24]:
# Number of entries
print(df.shape[0])
print(df_dedup.shape[0])

(636061, 29)
(125842, 29)


In [28]:
# Number of blank coordinate entries
df_blank_coords = df[df['lat'] == '']
print(df_blank_coords.shape[0])

df_dedup_blank_coords = df_dedup[df_dedup['lat'] == '']
print(df_dedup_blank_coords.shape[0])

(299068, 29)
(56086, 29)


In [14]:
import inspect 
for i in inspect.getmembers(grp_count):
    if not i[0].startswith('_'):
        if not inspect.ismethod(i[1]): 
            print(i)

('T', bus_idx                                                                                            FOLDERYEAR
!Exotics BySIYTHE Corporation_BC_Design Company_170_422_RICHARDS ST_Vancouver_V6B 2Z4_CA_          18            1
                                                                                                   19            1
#1 Antique Warehouse Furnishings Ltd_BC_Antique Dealer__226_SW MARINE DRIVE_Vancouver_V5X 2R5_CA_  13            1
                                                                                                   14            1
                                                                                                   15            1
                                                                                                                ..
xMatters Inc_BC_Software_1450_700_W PENDER ST_Vancouver_V6C 1G8_CA_                                19            1
xMatters Inc_BC_Software_200_510_BURRARD ST_Vancouver_V6C 3A8_CA_              

In [19]:
# Greater Vancouver Floating Home Co-Op
gvfhc_df = df[df['business_name'] == 'Greater Vancouver Floating Home Co-Op']
gvfhc_df.to_csv('/home/jovyan/ODBiz/1-PreProcessing/vancouver_dedup/Greater Vancouver Floating Home Co-Op.csv')
